In [1]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-
import os

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch import nn

from dataloader import svhn
from models import DenseNet121
from src import freeze_influence, hessians, selection, utils

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
def load_net(net, path):
    assert os.path.isfile(path), "Error: no checkpoint file found!"
    checkpoint = torch.load(path)
    net.load_state_dict(checkpoint["net"])
    return net


def save_net(net, path):
    dir, filename = os.path.split(path)
    if not os.path.isdir(dir):
        os.makedirs(dir)

    state = {
        "net": net.state_dict(),
    }
    torch.save(state, path)


def test(net, dataloader, criterion, label, include):
    net.eval()
    with torch.no_grad():
        net_loss = 0
        correct = 0
        total = 0
        num_data = 0
        for _, (inputs, targets) in enumerate(dataloader):
            if include:
                idx = targets == label
            else:
                idx = targets != label
            inputs = inputs[idx]
            targets = targets[idx]
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            net_loss += loss * len(inputs)
            num_data +=  len(inputs)

            total += targets.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()

        accuracy = correct / total * 100
        net_loss /= num_data
        return net_loss, accuracy


def get_full_param_list(net):
    """
    Return a list of parameter indices in flatten network.
    Warning: this function only provides indices of params when the param i) has requires_grad=True and 2) belongs to nn.Linear or nn.Conv2d
    """

    index_list = np.array([], dtype=int)
    start_index = 0
    for module in net.modules():
        if not list(module.children()) == []:
            continue

        num_param = sum(p.numel() for p in module.parameters() if p.requires_grad)
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            module_index_list = np.arange(num_param, dtype=int) + start_index
            index_list = np.append(index_list, module_index_list)

        start_index += num_param

    return index_list


def projected_influence(net, index_list, total_loss, target_loss, tol, step):
    full_param_list = get_full_param_list(net)
    influence = hessians.partial_influence(
        full_param_list, target_loss, total_loss, net, tol=tol, step=step
    )
    idx = np.isin(full_param_list, index_list)
    return influence[idx], full_param_list[idx]

In [5]:
torch.manual_seed(0)
# np.random.seed(0)
net = DenseNet121().to(device)
net_name = net.__class__.__name__

if device == "cuda":
    cudnn.benchmark = True

net_path = f"../checkpoints/Figure_4/{net_name}/cross_entropy/ckpt_0.0.pth"
net = load_net(net, net_path)

net.eval()
num_param = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(
    f"==> Building {net_name} finished. "
    + f"\n    Number of parameters: {num_param}"
)

criterion = nn.CrossEntropyLoss()

# Data
print("==> Preparing data..")
batch_size = 256
num_workers = 12
num_sample_batch = 1
num_target_sample = 256

data_loader = svhn.SVHNDataLoader(batch_size, num_workers)
train_loader, test_loader = data_loader.get_data_loaders()

==> Building DenseNet finished. 
    Number of parameters: 6956298
==> Preparing data..
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [4]:
loss, acc = test(net, test_loader, criterion, 11, False)
print(
    f"Original loss and acc : {loss:.4f}, {acc:.2f}%"
)

Original loss and acc : 0.1667, 95.37%


In [6]:
inputs_list = list()
targets_list = list()

for batch_idx, (inputs, targets) in enumerate(train_loader):
    if batch_idx < num_sample_batch:
        inputs_list.append(inputs)
        targets_list.append(targets)
    else:
        break
        
data = list()
target = list()
for batch_idx, (data_raw, target_raw) in enumerate(train_loader):
    idx = target_raw == 8
    data_raw = data_raw[idx]
    target_raw = target_raw[idx]
    data.append(data_raw)
    target.append(target_raw)
data = torch.cat(data)
target = torch.cat(target)
sample_idx = np.random.choice(len(data), num_target_sample, replace=False)
sample_data = data[sample_idx]
sample_target = target[sample_idx]

In [7]:
ratio_list = [.1, .3, .5]
tol_list = [2e-8, 1.5e-8, 1e-8]

In [8]:
for param_ratio, tol in zip(ratio_list, tol_list):
    print(f"Ratio: {param_ratio*100}%, tol: {tol}")
    for i in range(3):
        net = load_net(net, net_path)
        total_loss = 0
        for inputs, targets in zip(inputs_list, targets_list):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss

        total_loss /= num_sample_batch

        # Make hooks
        percentage = 0.1
        net_parser = selection.TopNActivations(net, param_ratio)
        net_parser.register_hooks()

        target_loss = (
            criterion(net(sample_data.to(device)), sample_target.to(device))
            * len(data)
            / len(train_loader.dataset)
        )
        target_loss.backward(retain_graph=True)
        data_ratio = len(train_loader.dataset) / (len(train_loader.dataset) - len(data))

        index_list = net_parser.get_parameters()
        net_parser.remove_hooks()

        if i == 0:
            influence = hessians.partial_influence(
                index_list, target_loss, total_loss, net, tol=tol, step=5
            )
            if_name = "GIF"
        elif i == 1:
            influence = freeze_influence.freeze_influence(
                index_list, target_loss, total_loss, net, tol=tol, step=5
            )
            if_name = "FIF"
        else:
            influence, index_list = projected_influence(
                net, index_list, total_loss, target_loss, tol=tol/2, step=5
            )
            if_name = "PIF"
            
        for scale in range(10,801):
            scale /= 5
            utils.update_network(net, influence * scale, index_list)
#             save_path = f"../checkpoints/Figure_4/{if_name}/{net_name}/cross_entropy/ckpt_0.0_{param_ratio}.pth"
#             save_net(net, save_path)

            self_loss, self_acc = test(net, test_loader, criterion, 8, True)
            exclusive_loss, exclusive_acc = test(net, test_loader, criterion, 8, False)
            print(
                f"{if_name} Self: {self_loss:.4f} {self_acc:2.2f}% | Exclusive loss: {exclusive_loss:.4f}, {exclusive_acc:2.2f}% | Scale: {scale} | Score: {self_loss-exclusive_loss:.4f}, {exclusive_acc-self_acc:.2f}"
            )
            if exclusive_acc <= 70 or self_acc <= 0.01:
                break
        print("")
    print("")

Ratio: 10.0%, tol: 2e-08


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 11.77 GiB total capacity; 10.21 GiB already allocated; 113.19 MiB free; 10.42 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
retrained_net = DenseNet121().to(device)
net_name = retrained_net.__class__.__name__
net_path = f"../checkpoints/Figure_4/{net_name}/cross_entropy/ckpt_0.0_retrained.pth"
retrained_net = load_net(retrained_net, net_path)
flatten = False

loss, acc = test(retrained_net, test_loader, criterion, 11, False)
print(
    f"Original loss and acc : {loss:.4f}, {acc:.2f}%"
)
self_loss, self_acc = test(retrained_net, test_loader, criterion, 8, True)
exclusive_loss, exclusive_acc = test(retrained_net, test_loader, criterion, 8, False)
print(
    f"Retrained model \t Self: {self_loss:.2f} {self_acc:2.2f}% | Exclusive loss: {exclusive_loss:.2f}, {exclusive_acc:2.2f}%"
)

In [ ]:
def calculate_inf_err(model1, model2):
    l2_norm = 0.0
    for param1, param2 in zip(model1.parameters(), model2.parameters()):
        diff = param1 - param2
        l2_norm += torch.norm(diff, p=2) ** 2
    l2_norm = torch.sqrt(l2_norm)
    return l2_norm

for param_ratio, tol, scale in zip(ratio_list, tol_list, scale_list):
    for i in range(3):
        if i == 0:
            if_name = "GIF"
        elif i == 1:
            if_name = "FIF"
        else:
            if_name = "PIF"

        save_path = f"../checkpoints/Figure_4/{if_name}/{net_name}/cross_entropy/ckpt_0.0_{param_ratio}.pth"
        net = load_net(net, save_path)
        influence_err = calculate_inf_err(net, retrained_net)
        self_loss, self_acc = test(net, test_loader, criterion, 8, True)
        exclusive_loss, exclusive_acc = test(net, test_loader, criterion, 8, False)
        if if_name == "PIF":
            if_name += "\t"
        print(
            f"  {if_name}\t Self: {self_loss:.2f} {self_acc:02.2f}% | Exclusive loss: {exclusive_loss:.2f}, {exclusive_acc:2.2f}% | Influence error: {influence_err}"
        )
    print("")

## 